# Correlation analysis between PyRadiomics and FMCIB imaging features

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import sys; sys.path.insert(0, '..')
from scripts.python.readii_analysis.data.helpers import (
    loadFeatureFilesFromImageTypes, 
    loadImageDatasetConfig,
    savePlotFigure)
from scripts.python.readii_analysis.analyze.correlation_functions import (
    getFeatureCorrelations, 
    plotCrossCorrelationHeatmap)

In [6]:
dataset_name = "RADCURE"
extracted_feature_dir = "train_test_split/train_features"

PROC_DATA_PATH = "../../procdata/"
RESULTS_DATA_PATH = "../../results/"
CONFIG_DIR_PATH = "../config/"
CORRELATION_METHOD = "pearson"
CORR_COLOR_MAP = "viridis"

In [3]:
config = loadImageDatasetConfig(dataset_name, CONFIG_DIR_PATH)

In [7]:
output_dir = os.path.join(RESULTS_DATA_PATH, dataset_name, "pyradiomics_vs_fmcib")
os.makedirs(output_dir, exist_ok=True)

In [5]:
pyradiomics_features_dir = os.path.join(PROC_DATA_PATH, dataset_name, "radiomic", extracted_feature_dir)
fmcib_features_dir = os.path.join(PROC_DATA_PATH, dataset_name, "deep_learning", extracted_feature_dir)

pyradiomic_feature_sets = loadFeatureFilesFromImageTypes(pyradiomics_features_dir, 
                                                         image_types=config["image_types"], 
                                                         drop_labels=True, 
                                                         labels_to_drop=["patient_ID","survival_time_in_years","survival_event_binary"])

fmcib_feature_sets = loadFeatureFilesFromImageTypes(fmcib_features_dir, 
                                                    image_types=config["image_types"], 
                                                    drop_labels=True, 
                                                    labels_to_drop=["patient_ID","survival_time_in_years","survival_event_binary"])

In [ ]:
for image_type in config["image_types"]:
    print(f"Calculating {image_type} correlations")
    pyrad_features = pyradiomic_feature_sets[image_type]
    fmcib_features = fmcib_feature_sets[image_type]

    feature_correlation_matrix = getFeatureCorrelations(vertical_features = pyrad_features,
                                                        horizontal_features = fmcib_features,
                                                        method="pearson",
                                                        vertical_feature_name="pyradiomics",
                                                        horizontal_feature_name="fmcib")
    print()
    
    pyrad_vs_fmcib_corr_plot = plotCrossCorrelationHeatmap(correlation_matrix=feature_correlation_matrix,
                                                           num_vertical_features=len(pyrad_features),
                                                           vertical_feature_name="PyRadiomics",
                                                           horizontal_feature_name="FMCIB",
                                                           cmap=CORR_COLOR_MAP,
                                                           correlation_method="Pearson",
                                                           dataset_name=dataset_name
                                                           )
    plt.close('all')
    savePlotFigure(pyrad_vs_fmcib_corr_plot,
                   plot_name=f"{dataset_name}_Pearson_corr_pyrad_vs_fmcib_{image_type}_plot.png",
                   output_dir_path=output_dir)

TypeError: plotCrossCorrelationHeatmap() got an unexpected keyword argument 'corr_cmap'